In [1]:
# Install required packages
!pip install shap lime torch pandas numpy scikit-learn matplotlib seaborn plotly

In [2]:
# Import required libraries
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

c:\Users\Shraddha\OneDrive\Desktop\TT\Trace_Transformer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries imported successfully!


Load the data files in the next cell:

In [ ]:
# Load the preprocessed datasets
# Note: Make sure you have both balanced dataset files in your workspace
df_borderline = pd.read_csv('balanced_dataset_borderline_smote.csv')
df_adasyn = pd.read_csv('balanced_dataset_adasyn.csv')

# Function to prepare data
def prepare_data(df, test_size=0.2, random_state=42):
    X = df.drop('label', axis=1)
    y = df['label']
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    X_test_tensor = torch.FloatTensor(X_test_scaled)
    y_train_tensor = torch.LongTensor(y_train.values)
    y_test_tensor = torch.LongTensor(y_test.values)
    
    return {
        'X_train': X_train, 'X_test': X_test,
        'y_train': y_train, 'y_test': y_test,
        'X_train_scaled': X_train_scaled, 'X_test_scaled': X_test_scaled,
        'X_train_tensor': X_train_tensor, 'X_test_tensor': X_test_tensor,
        'y_train_tensor': y_train_tensor, 'y_test_tensor': y_test_tensor,
        'scaler': scaler, 'feature_names': X.columns
    }

# Prepare both datasets
data_borderline = prepare_data(df_borderline)
data_adasyn = prepare_data(df_adasyn)

print("✅ Data loaded and prepared successfully!")
print("\nBorderline-SMOTE Dataset:")
print(f"Training set shape: {data_borderline['X_train_scaled'].shape}")
print(f"Test set shape: {data_borderline['X_test_scaled'].shape}")
print("\nADASYN Dataset:")
print(f"Training set shape: {data_adasyn['X_train_scaled'].shape}")
print(f"Test set shape: {data_adasyn['X_test_scaled'].shape}")

# We'll use these variables for compatibility with existing code
# Default to Borderline-SMOTE data
X_train = data_borderline['X_train']
X_test = data_borderline['X_test']
y_train = data_borderline['y_train']
y_test = data_borderline['y_test']
X_train_scaled = data_borderline['X_train_scaled']
X_test_scaled = data_borderline['X_test_scaled']
X_train_tensor = data_borderline['X_train_tensor']
X_test_tensor = data_borderline['X_test_tensor']
y_train_tensor = data_borderline['y_train_tensor']
y_test_tensor = data_borderline['y_test_tensor']
scaler = data_borderline['scaler']

# Interpretability Analysis

In [ ]:
#Setting up interpretability for FTTransformer
class InterpretableFTTransformer(nn.Module):
    def __init__(self, num_features, num_classes, dim=128, depth=6, heads=8, dropout=0.1):
        super().__init__()
        
        self.dim = dim
        self.feature_embedding = nn.Linear(num_features, dim)
        self.pos_encoding = nn.Parameter(torch.zeros(1, 1, dim))
        
        # Store attention weights for visualization
        self.attention_weights = []
        
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=dim,
                nhead=heads,
                dim_feedforward=dim*4,
                dropout=dropout,
                batch_first=True
            ) for _ in range(depth)
        ])
        
        self.classifier = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, dim//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim//2, num_classes)
        )
        
        self.dropout = nn.Dropout(dropout)
        self._init_weights()
    
    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()
    
    def forward(self, x, return_attention=False):
        self.attention_weights = []  # Reset attention weights
        
        batch_size = x.shape[0]
        x = self.feature_embedding(x)
        x = x.unsqueeze(1)
        x = x + self.pos_encoding
        
        for transformer_layer in self.transformer_layers:
            # Store attention weights for visualization
            if return_attention:
                self.attention_weights.append(
                    transformer_layer.self_attn(x, x, x)[1]
                )
            x = transformer_layer(x)
        
        x = x.mean(dim=1)
        output = self.classifier(x)
        
        if return_attention:
            return output, self.attention_weights
        return output

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = InterpretableFTTransformer(
    num_features=X_train.shape[1],
    num_classes=len(np.unique(y_train)),
).to(device)

print("✅ Model initialized successfully!")

In [ ]:
# Training settings
num_epochs = 10
batch_size = 32
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # Print epoch statistics
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor.to(device))
    _, predicted = torch.max(test_outputs, 1)
    correct = (predicted == y_test_tensor.to(device)).sum().item()
    accuracy = correct / len(y_test_tensor)

print(f"\n✅ Model training completed!")
print(f"Test Accuracy: {accuracy:.4f}")

# SHAP Analysis

In [ ]:
#SHAP (SHapley Additive exPlanations) analysis to understand feature importance and model decisions.
class ShapExplainer:
    def __init__(self, model, feature_names):
        self.model = model
        self.feature_names = feature_names
        
    def explain_instance(self, instance, background_data):
        # Create a PyTorch model wrapper for SHAP
        def model_predict(x):
            x_tensor = torch.FloatTensor(x).to(device)
            with torch.no_grad():
                return self.model(x_tensor).cpu().numpy()
        
        # Initialize the SHAP explainer
        explainer = shap.KernelExplainer(model_predict, background_data)
        
        # Calculate SHAP values
        shap_values = explainer.shap_values(instance)
        
        return shap_values
    
    def plot_feature_importance(self, instance, background_data):
        shap_values = self.explain_instance(instance, background_data)
        
        # Create summary plot
        plt.figure(figsize=(10, 6))
        shap.summary_plot(
            shap_values[1] if isinstance(shap_values, list) else shap_values,
            instance,
            feature_names=self.feature_names,
            show=False
        )
        plt.title("SHAP Feature Importance")
        plt.tight_layout()
        plt.show()

# Initialize SHAP explainer
explainer = ShapExplainer(model, X_train.columns)

# Get background data for SHAP (using a subset of training data)
background_data = X_train_scaled[:100]  # Using first 100 samples as background

# Example: Explain a single instance
instance_to_explain = X_test_scaled[0:1]  # First test instance
explainer.plot_feature_importance(instance_to_explain, background_data)

print("✅ SHAP analysis completed!")

# Attention Visualization

In [ ]:
#Visualize the attention weights from the transformer model to understand which features the model focuses on.
class AttentionVisualizer:
    def __init__(self, model, feature_names):
        self.model = model
        self.feature_names = feature_names
    
    def get_attention_weights(self, input_data):
        input_tensor = torch.FloatTensor(input_data).to(device)
        with torch.no_grad():
            _, attention_weights = self.model(input_tensor, return_attention=True)
        return attention_weights
    
    def plot_attention_heatmap(self, input_data, layer_idx=0):
        attention_weights = self.get_attention_weights(input_data)
        
        # Get attention weights for specified layer
        layer_attention = attention_weights[layer_idx][0].cpu().numpy()
        
        # Create heatmap
        plt.figure(figsize=(12, 8))
        sns.heatmap(
            layer_attention,
            xticklabels=self.feature_names,
            yticklabels=self.feature_names,
            cmap='viridis'
        )
        plt.title(f'Attention Weights - Layer {layer_idx + 1}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    
    def plot_feature_attention_summary(self, input_data):
        attention_weights = self.get_attention_weights(input_data)
        
        # Average attention across all layers
        avg_attention = torch.mean(
            torch.stack([w[0] for w in attention_weights]), 
            dim=0
        ).cpu().numpy()
        
        # Calculate mean attention per feature
        feature_attention = avg_attention.mean(axis=0)
        
        # Create bar plot
        plt.figure(figsize=(12, 6))
        plt.bar(self.feature_names, feature_attention)
        plt.title('Average Feature Attention')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

# Initialize visualizer
visualizer = AttentionVisualizer(model, X_train.columns)

# Visualize attention for a single instance
instance_to_visualize = X_test_scaled[0:1]
visualizer.plot_attention_heatmap(instance_to_visualize)
visualizer.plot_feature_attention_summary(instance_to_visualize)

print("✅ Attention visualization completed!")

# Counterfactual Explanations

In [ ]:
#Generate counterfactual explanations to show how predictions would change with different inputs.
class CounterfactualGenerator:
    def __init__(self, model, feature_names, scaler):
        self.model = model
        self.feature_names = feature_names
        self.scaler = scaler
    
    def generate_counterfactuals(self, instance, target_class, num_samples=1000, eps=0.1):
        # Generate random perturbations
        noise = np.random.normal(0, eps, size=(num_samples, instance.shape[1]))
        candidates = instance + noise
        
        # Get predictions for all candidates
        candidates_tensor = torch.FloatTensor(candidates).to(device)
        with torch.no_grad():
            predictions = self.model(candidates_tensor)
            probs = torch.softmax(predictions, dim=1).cpu().numpy()
        
        # Find the closest counterfactual
        target_probs = probs[:, target_class]
        best_idx = np.argmax(target_probs)
        
        return candidates[best_idx], target_probs[best_idx]
    
    def explain_counterfactual(self, original_instance, counterfactual):
        # Calculate and sort feature differences
        diff = counterfactual - original_instance
        feature_diffs = list(zip(self.feature_names, diff[0]))
        feature_diffs.sort(key=lambda x: abs(x[1]), reverse=True)
        
        # Create visualization
        plt.figure(figsize=(12, 6))
        features, values = zip(*[(f, v) for f, v in feature_diffs if abs(v) > 0.01])
        
        plt.bar(features, values)
        plt.title('Feature Changes for Counterfactual Example')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('Change in Feature Value')
        plt.tight_layout()
        plt.show()
        
        return feature_diffs

# Initialize counterfactual generator
counterfactual_gen = CounterfactualGenerator(model, X_train.columns, scaler)

# Generate counterfactual for a test instance
instance_to_explain = X_test_scaled[0:1]
target_class = 1  # Desired target class
counterfactual, confidence = counterfactual_gen.generate_counterfactuals(
    instance_to_explain, target_class
)

# Explain the counterfactual
feature_differences = counterfactual_gen.explain_counterfactual(
    instance_to_explain, counterfactual
)

print("✅ Counterfactual explanation generated!")
print(f"Confidence in target class: {confidence:.3f}")

# Zero-Trust Implementation

In [ ]:
#Implement zero-trust security measures for the ML pipeline.
class ZeroTrustWrapper:
    def __init__(self, model, feature_names, scaler):
        self.model = model
        self.feature_names = feature_names
        self.scaler = scaler
        self.feature_stats = self._compute_feature_stats()
    
    def _compute_feature_stats(self):
        """Compute statistics for input validation"""
        return {
            'mean': np.mean(X_train, axis=0),
            'std': np.std(X_train, axis=0),
            'min': np.min(X_train, axis=0),
            'max': np.max(X_train, axis=0)
        }
    
    def _validate_input(self, X):
        """Validate input data"""
        validations = []
        
        # Check for missing values
        if np.any(np.isnan(X)):
            validations.append("Input contains missing values")
        
        # Check for infinity values
        if np.any(np.isinf(X)):
            validations.append("Input contains infinity values")
        
        # Check for out-of-bounds values (using z-score)
        z_scores = np.abs((X - self.feature_stats['mean']) / self.feature_stats['std'])
        if np.any(z_scores > 5):  # More than 5 standard deviations
            validations.append("Input contains extreme values")
        
        return validations
    
    def _check_model_confidence(self, probabilities, threshold=0.8):
        """Check if model confidence is above threshold"""
        max_prob = np.max(probabilities)
        return max_prob >= threshold, max_prob
    
    def predict(self, X, return_diagnostics=False):
        # Input validation
        validation_results = self._validate_input(X)
        
        # Transform input
        X_scaled = self.scaler.transform(X)
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        
        # Model prediction with confidence check
        with torch.no_grad():
            logits = self.model(X_tensor)
            probabilities = torch.softmax(logits, dim=1).cpu().numpy()
        
        confident, confidence = self._check_model_confidence(probabilities)
        predictions = np.argmax(probabilities, axis=1)
        
        if return_diagnostics:
            diagnostics = {
                'validation_results': validation_results,
                'confidence': confidence,
                'is_confident': confident,
                'probabilities': probabilities
            }
            return predictions, diagnostics
        
        return predictions

# Initialize zero-trust wrapper
zero_trust_model = ZeroTrustWrapper(model, X_train.columns, scaler)

# Test prediction with diagnostics
test_instance = X_test[:1]
predictions, diagnostics = zero_trust_model.predict(test_instance, return_diagnostics=True)

print("✅ Zero-trust prediction completed!")
print("\nDiagnostics:")
print(f"Validation results: {diagnostics['validation_results'] if diagnostics['validation_results'] else 'All validations passed'}")
print(f"Confidence: {diagnostics['confidence']:.3f}")
print(f"Is confident: {diagnostics['is_confident']}")
print(f"Prediction: {predictions[0]}")

# Explainable AI Wrapper

In [ ]:
class ExplainableAIWrapper:
    def __init__(self, model, data_borderline, data_adasyn):
        self.model = model
        self.data_borderline = data_borderline
        self.data_adasyn = data_adasyn
        
        # Initialize components for both datasets
        self.components = {
            'borderline': {
                'zero_trust': ZeroTrustWrapper(model, data_borderline['feature_names'], data_borderline['scaler']),
                'shap_explainer': ShapExplainer(model, data_borderline['feature_names']),
                'attention_viz': AttentionVisualizer(model, data_borderline['feature_names']),
                'counterfactual_gen': CounterfactualGenerator(model, data_borderline['feature_names'], data_borderline['scaler'])
            },
            'adasyn': {
                'zero_trust': ZeroTrustWrapper(model, data_adasyn['feature_names'], data_adasyn['scaler']),
                'shap_explainer': ShapExplainer(model, data_adasyn['feature_names']),
                'attention_viz': AttentionVisualizer(model, data_adasyn['feature_names']),
                'counterfactual_gen': CounterfactualGenerator(model, data_adasyn['feature_names'], data_adasyn['scaler'])
            }
        }
    
    def explain_prediction(self, instance, dataset_type='borderline', background_data=None):
        """Generate a comprehensive explanation for a prediction"""
        components = self.components[dataset_type]
        data = self.data_borderline if dataset_type == 'borderline' else self.data_adasyn
        
        print(f"\n{'='*20} Analysis using {dataset_type.upper()} Dataset {'='*20}")
        
        # Get prediction with zero-trust
        pred, diagnostics = components['zero_trust'].predict(instance, return_diagnostics=True)
        
        print("\n1. Zero-Trust Diagnostics:")
        print(f"Prediction: {pred[0]}")
        print(f"Confidence: {diagnostics['confidence']:.3f}")
        print(f"Validation Results: {diagnostics['validation_results'] if diagnostics['validation_results'] else 'All validations passed'}")
        print("\n" + "="*50 + "\n")
        
        # Get SHAP explanation
        print("2. SHAP Feature Importance:")
        if background_data is None:
            background_data = data['X_train_scaled'][:100]
        components['shap_explainer'].plot_feature_importance(
            data['scaler'].transform(instance), 
            background_data
        )
        print("\n" + "="*50 + "\n")
        
        # Get attention visualization
        print("3. Attention Analysis:")
        components['attention_viz'].plot_attention_heatmap(
            data['scaler'].transform(instance)
        )
        components['attention_viz'].plot_feature_attention_summary(
            data['scaler'].transform(instance)
        )
        print("\n" + "="*50 + "\n")
        
        # Generate counterfactual
        print("4. Counterfactual Explanation:")
        target_class = 1 if pred[0] == 0 else 0  # Opposite class
        counterfactual, confidence = components['counterfactual_gen'].generate_counterfactuals(
            data['scaler'].transform(instance),
            target_class
        )
        components['counterfactual_gen'].explain_counterfactual(
            data['scaler'].transform(instance),
            counterfactual
        )
        
        return {
            'prediction': pred[0],
            'diagnostics': diagnostics,
            'counterfactual_confidence': confidence
        }
    
    def compare_explanations(self, instance):
        """Compare explanations between Borderline-SMOTE and ADASYN datasets"""
        borderline_results = self.explain_prediction(instance, 'borderline')
        print("\n" + "="*50 + "\nComparing with ADASYN dataset:\n" + "="*50 + "\n")
        adasyn_results = self.explain_prediction(instance, 'adasyn')
        
        # Compare predictions and confidence
        print("\n📊 Comparison Summary:")
        print("-" * 40)
        print("Predictions:")
        print(f"• Borderline-SMOTE: {borderline_results['prediction']} (conf: {borderline_results['diagnostics']['confidence']:.3f})")
        print(f"• ADASYN: {adasyn_results['prediction']} (conf: {adasyn_results['diagnostics']['confidence']:.3f})")
        
        # Compare counterfactual confidence
        print("\nCounterfactual Generation:")
        print(f"• Borderline-SMOTE confidence: {borderline_results['counterfactual_confidence']:.3f}")
        print(f"• ADASYN confidence: {adasyn_results['counterfactual_confidence']:.3f}")
        
        return {
            'borderline': borderline_results,
            'adasyn': adasyn_results
        }

# Initialize explainable AI wrapper with both datasets
xai_wrapper = ExplainableAIWrapper(model, data_borderline, data_adasyn)

# Generate comprehensive explanation for a test instance using both datasets
test_instance = X_test[:1]
comparison_results = xai_wrapper.compare_explanations(test_instance)

print("\n✅ Comparative analysis completed!")

# Online Learning Integration

Implement online learning capabilities to update the model with new data.

In [ ]:
class OnlineLearner:
    def __init__(self, model, scaler, buffer_size=1000):
        self.model = model
        self.scaler = scaler
        self.buffer_size = buffer_size
        self.buffer_X = []
        self.buffer_y = []
        
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        self.criterion = nn.CrossEntropyLoss()
    
    def update(self, X, y):
        """Update the model with new data"""
        # Add to buffer
        self.buffer_X.extend(X)
        self.buffer_y.extend(y)
        
        # Maintain buffer size
        if len(self.buffer_X) > self.buffer_size:
            self.buffer_X = self.buffer_X[-self.buffer_size:]
            self.buffer_y = self.buffer_y[-self.buffer_size:]
        
        # Prepare data
        X_scaled = self.scaler.transform(np.array(self.buffer_X))
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        y_tensor = torch.LongTensor(self.buffer_y).to(device)
        
        # Update model
        self.model.train()
        self.optimizer.zero_grad()
        outputs = self.model(X_tensor)
        loss = self.criterion(outputs, y_tensor)
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
    
    def evaluate_performance(self, X_test, y_test):
        """Evaluate current model performance"""
        self.model.eval()
        X_scaled = self.scaler.transform(X_test)
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        
        with torch.no_grad():
            outputs = self.model(X_tensor)
            _, predicted = torch.max(outputs.data, 1)
        
        predicted = predicted.cpu().numpy()
        return classification_report(y_test, predicted)

# Initialize online learner
online_learner = OnlineLearner(model, scaler)

# Simulate online learning with new data
# For demonstration, we'll use some test data as new data
new_data_X = X_test[:10]
new_data_y = y_test[:10]

# Update model
loss = online_learner.update(new_data_X, new_data_y)

# Evaluate updated model
performance_report = online_learner.evaluate_performance(X_test, y_test)

print("✅ Online learning update completed!")
print(f"Update loss: {loss:.4f}\n")
print("Updated Model Performance:")
print(performance_report)

# Project Summary and Conclusions

This section provides a comprehensive summary of the Trace Transformer project's interpretability and innovation components.

In [ ]:
# Create a comprehensive project summary comparing both approaches
def generate_project_summary(model, data_borderline, data_adasyn):
    summary = {
        'model_performance': {
            'borderline': {},
            'adasyn': {}
        },
        'interpretability_metrics': {
            'borderline': {},
            'adasyn': {}
        },
        'comparative_analysis': {}
    }
    
    # Function to evaluate model performance
    def evaluate_dataset(data, dataset_name):
        model.eval()
        with torch.no_grad():
            test_tensor = torch.FloatTensor(data['X_test_scaled']).to(device)
            outputs = model(test_tensor)
            _, predicted = torch.max(outputs, 1)
            probabilities = torch.softmax(outputs, dim=1).cpu().numpy()
        
        predictions = predicted.cpu().numpy()
        test_labels = data['y_test'].values
        confidence_scores = np.max(probabilities, axis=1)
        
        # Calculate metrics
        accuracy = (predictions == test_labels).mean()
        avg_confidence = confidence_scores.mean()
        high_conf_preds = (confidence_scores > 0.9).mean()
        
        # Get feature importance from attention weights
        visualizer = AttentionVisualizer(model, data['feature_names'])
        sample_instance = data['X_test_scaled'][0:1]
        attention_weights = visualizer.get_attention_weights(sample_instance)
        avg_attention = torch.mean(torch.stack([w[0] for w in attention_weights]), dim=0).cpu().numpy()
        feature_importance = avg_attention.mean(axis=0)
        
        # Get top important features
        feature_importance_dict = dict(zip(data['feature_names'], feature_importance))
        top_features = dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:5])
        
        return {
            'accuracy': accuracy,
            'average_confidence': avg_confidence,
            'high_confidence_predictions': high_conf_preds,
            'top_features': top_features,
            'attention_stability': np.std(feature_importance)
        }
    
    # Evaluate both datasets
    borderline_metrics = evaluate_dataset(data_borderline, 'Borderline-SMOTE')
    adasyn_metrics = evaluate_dataset(data_adasyn, 'ADASYN')
    
    # Store results
    for dataset_name, metrics in [('borderline', borderline_metrics), ('adasyn', adasyn_metrics)]:
        summary['model_performance'][dataset_name] = {
            'accuracy': metrics['accuracy'],
            'average_confidence': metrics['average_confidence'],
            'high_confidence_predictions': metrics['high_confidence_predictions']
        }
        summary['interpretability_metrics'][dataset_name] = {
            'top_important_features': metrics['top_features'],
            'attention_pattern_stability': metrics['attention_stability']
        }
    
    # Comparative analysis
    summary['comparative_analysis'] = {
        'accuracy_difference': abs(borderline_metrics['accuracy'] - adasyn_metrics['accuracy']),
        'confidence_difference': abs(borderline_metrics['average_confidence'] - adasyn_metrics['average_confidence']),
        'shared_top_features': len(
            set(borderline_metrics['top_features'].keys()) & 
            set(adasyn_metrics['top_features'].keys())
        )
    }
    
    # Create formatted report
    print("="*60)
    print("🔍 TRACE TRANSFORMER PROJECT SUMMARY - COMPARATIVE ANALYSIS")
    print("="*60)
    
    for dataset in ['borderline', 'adasyn']:
        print(f"\n📊 {dataset.upper()} Dataset Performance:")
        print(f"   • Accuracy: {summary['model_performance'][dataset]['accuracy']:.4f}")
        print(f"   • Average Confidence: {summary['model_performance'][dataset]['average_confidence']:.4f}")
        print(f"   • High Confidence Predictions: {summary['model_performance'][dataset]['high_confidence_predictions']*100:.2f}%")
        
        print(f"\n🎯 Top Important Features ({dataset.upper()}):")
        for feature, importance in summary['interpretability_metrics'][dataset]['top_important_features'].items():
            print(f"   • {feature}: {importance:.4f}")
    
    print("\n? Comparative Analysis:")
    print(f"   • Accuracy Difference: {summary['comparative_analysis']['accuracy_difference']:.4f}")
    print(f"   • Confidence Difference: {summary['comparative_analysis']['confidence_difference']:.4f}")
    print(f"   • Shared Top Features: {summary['comparative_analysis']['shared_top_features']}/5")
    
    print("\n? Innovation Components Status:")
    print("   • Zero-trust Implementation: Active for both datasets")
    print("   • Explainable AI Framework: Enhanced with comparative analysis")
    print("   • Online Learning: Adaptive to both sampling methods")
    print("   • Attention Visualization: Comparative feature importance analysis")
    
    print("\n🚀 Key Findings:")
    if summary['model_performance']['borderline']['accuracy'] > summary['model_performance']['adasyn']['accuracy']:
        better_method = "Borderline-SMOTE"
    else:
        better_method = "ADASYN"
    print(f"   • {better_method} shows better overall performance")
    print(f"   • {summary['comparative_analysis']['shared_top_features']} shared important features between methods")
    print("   • Both methods maintain robust interpretability")
    
    return summary

# Generate and display the comparative summary
summary = generate_project_summary(model, data_borderline, data_adasyn)

print("\n" + "="*60)
print("✅ Comparative Project Summary Generated Successfully!")
print("="*60)